### Решение Задания 1: Основы `\copy` (Экспорт и Импорт)

**1. Экспорт:**

Откройте терминал и подключитесь к вашей базе данных `northwind` с помощью `psql`.

```bash
psql -d northwind
```

Теперь выполните команду экспорта. Мы явно перечисляем столбцы, чтобы исключить `discontinued`.

```sql
\copy (SELECT product_id, product_name, supplier_id, category_id, quantity_per_unit, unit_price, units_in_stock, units_on_order, reorder_level FROM products) TO 'products_export.csv' WITH (FORMAT CSV, HEADER, DELIMITER ';', NULL '(not set)');
```

**2. Импорт и проверка:**

Сначала создаем таблицу-копию:

```sql
CREATE TABLE products_copy (LIKE products INCLUDING ALL);
```

Затем импортируем данные из файла, который мы только что создали. Настройки (`DELIMITER`, `NULL` и т.д.) должны в точности соответствовать тем, что использовались при экспорте.

```sql
\copy products_copy FROM 'products_export.csv' WITH (FORMAT CSV, HEADER, DELIMITER ';', NULL '(not set)');
```

**Проверка:**

Выполним подсчет строк в обеих таблицах. Результат должен быть одинаковым.

```sql
SELECT COUNT(*) FROM products;
--  count
-- -------
--     77
--(1 row)

SELECT COUNT(*) FROM products_copy;
--  count
-- -------
--     77
--(1 row)
```

-----

### Решение Задания 2: Продвинутый экспорт с использованием запроса

Эта команда объединяет три таблицы (`orders`, `order_details` и `products`), фильтрует заказы по 1997 году и выгружает результат напрямую в CSV файл.

```sql
\copy (
    SELECT
        o.order_id,
        o.order_date,
        p.product_name,
        od.quantity,
        od.unit_price
    FROM
        orders o
    JOIN
        order_details od ON o.order_id = od.order_id
    JOIN
        products p ON od.product_id = p.product_id
    WHERE
        EXTRACT(YEAR FROM o.order_date) = 1997
) TO 'sales_report_1997.csv' WITH (FORMAT CSV, HEADER);
```

-----

### Решение Задания 3: Импорт с обработкой ошибок (Staging Table)

**1. Создание Staging-таблицы:**

Мы используем `TEMP TABLE`, чтобы она автоматически удалилась после завершения сессии. Все поля имеют тип `TEXT` для "безопасной" загрузки любых данных.

```sql
CREATE TEMP TABLE suppliers_staging (
    supplier_name TEXT,
    contact_name TEXT,
    city TEXT,
    country TEXT,
    phone TEXT
);
```

**2. Загрузка "грязных" данных:**

Сохраните предоставленные CSV данные в файл `new_suppliers.csv` и выполните команду:

```sql
\copy suppliers_staging FROM 'new_suppliers.csv' WITH (FORMAT CSV);
```

**3. Очистка и вставка в основную таблицу:**

Это самый важный шаг. Мы используем `INSERT INTO ... SELECT`, чтобы преобразовать и отфильтровать данные "на лету".

```sql
INSERT INTO suppliers (company_name, contact_name, city, country, phone)
SELECT
    supplier_name,
    NULLIF(contact_name, ''),  -- Превращает пустую строку '' в NULL
    city,
    country,
    NULLIF(phone, '')          -- Превращает пустую строку '' в NULL
FROM
    suppliers_staging
WHERE
    country <> '??'; -- Отфильтровываем невалидную строку
```

**4. Проверка:**

Теперь посмотрим на последние добавленные записи в таблице `suppliers`. Они должны быть чистыми и корректными.

```sql
SELECT * FROM suppliers ORDER BY supplier_id DESC LIMIT 3;
```

Результат покажет три новые записи с корректными `NULL` значениями и без строки `"Invalid Data"`.

-----

### Решение Задания 4: Работа с `pg_dump` и `pg_restore`

**1. Создание бэкапа:**

Эта команда выполняется в вашем системном терминале (не в `psql`). Замените `your_user` на ваше имя пользователя PostgreSQL.

```bash
pg_dump -U your_user -Fc -f northwind.dump northwind
```

Вам будет предложено ввести пароль. После выполнения команды в текущей директории появится файл `northwind.dump`.

**2. Выборочное восстановление:**

Сначала создаем новую пустую базу данных:

```bash
createdb -U your_user northwind_partial_restore
```

Теперь используем `pg_restore` с флагом `-t` для каждой таблицы, которую хотим восстановить.

```bash
pg_restore -U your_user -d northwind_partial_restore -t employees -t customers northwind.dump
```

**Проверка:**

Подключитесь к новой базе и выведите список таблиц. Вы должны увидеть только `employees` и `customers`.

```bash
psql -d northwind_partial_restore
```

```sql
-- Внутри сессии psql
\dt
--             List of relations
-- Schema |   Name    | Type  |   Owner
-- --------+-----------+-------+----------
-- public | customers | table | your_user
-- public | employees | table | your_user
--(2 rows)
```

-----

### Решение Задания 5: Оптимизация производительности (Теоретический вопрос)

Вот мой пошаговый план действий для максимально быстрой загрузки 50 миллионов записей в таблицу `order_details`:

**Шаг 1: Начать транзакцию**

  * **Действие:** Выполнить команду `BEGIN;`.
  * **Почему это ускоряет:** Оборачивание всей операции в одну транзакцию минимизирует транзакционные издержки. Вместо 50 миллионов мелких транзакций СУБД выполняет одну большую, что значительно снижает нагрузку на запись в WAL (Write-Ahead Log).

**Шаг 2: Удалить все внешние ключи и индексы (кроме Primary Key)**

  * **Действие:** Выполнить `ALTER TABLE order_details DROP CONSTRAINT fk_orders;`, `ALTER TABLE order_details DROP CONSTRAINT fk_products;`, `DROP INDEX idx_quantity;`.
  * **Почему это ускоряет:** Это **самый важный** шаг.
      * **Внешние ключи:** Без них PostgreSQL не нужно для каждой из 50 млн строк выполнять проверку (lookup) в таблицах `orders` и `products`.
      * **Индексы:** При вставке каждой строки СУБД должна обновлять каждый существующий индекс. Это вызывает огромное количество медленных, случайных операций дискового ввода-вывода. Загрузка в "голую" таблицу — это быстрая последовательная запись.

**Шаг 3: Выполнить загрузку данных**

  * **Действие:** Использовать команду `COPY order_details FROM 'massive_file.csv' WITH (FORMAT CSV);`.
  * **Почему это ускоряет:** `COPY` — это самый быстрый нативный механизм загрузки данных в PostgreSQL, так как он работает на низком уровне с минимальными накладными расходами.

**Шаг 4: Воссоздать индексы и внешние ключи**

  * **Действие:** Увеличить `maintenance_work_mem` (`SET maintenance_work_mem = '2GB';`), затем выполнить `CREATE INDEX idx_quantity ON order_details(quantity);` и `ALTER TABLE order_details ADD CONSTRAINT ...;`.
  * **Почему это ускоряет:**
      * Построение индекса "с нуля" для уже заполненной таблицы — это одна эффективная операция последовательного чтения, которая выполняется гораздо быстрее, чем 50 миллионов мелких обновлений индекса.
      * Увеличение `maintenance_work_mem` позволяет PostgreSQL строить индекс в оперативной памяти, что многократно ускоряет процесс.
      * Проверка внешнего ключа после загрузки также выполняется одной массовой, более оптимизированной операцией для всей таблицы.

**Шаг 5: Завершить транзакцию**

  * **Действие:** Выполнить команду `COMMIT;`.
  * **Почему это важно:** Это атомарно фиксирует все изменения. Если на любом из предыдущих шагов произойдет сбой, вся транзакция откатится, и база данных останется в исходном, чистом состоянии.

-----
